In [ ]:
import pandas as pd
import datetime

# Create Master Dataset from ODRE

In [ ]:
master = pd.read_csv('/Users/ymilo/Desktop/Jedha/Final_Project/conso_elec_quotidienne_V2.csv')

In [ ]:
master['Date_index'] = pd.to_datetime(master['Date'])
master['year'] = master['Date_index'].dt.year
master['month'] = master['Date_index'].dt.month

# Create Oil price database

In [ ]:
oil = pd.read_excel('/Users/ymilo/Desktop/Jedha/Final_Project/RBRTEd (1).xls')
oil['Date_index'] = pd.to_datetime(oil['Date'])
oil_filtered = oil.loc[(oil['Date'] >= '2013-01-01') & (oil['Date'] < '2023-01-03')]
del oil_filtered['Date']

In [ ]:
master = master.merge(oil_filtered, on='Date_index', how='outer')

# Create Electricity price dataset

In [ ]:
electricity = pd.read_csv('/Users/ymilo/Desktop/Jedha/Final_Project/ten00117_linear.csv')

In [ ]:
electricity_fr = electricity[electricity['geo'] == 'FR']
electricity_fr = electricity_fr[electricity_fr['indic_en'] == 'MSHH']
electricity_fr.drop(electricity_fr.iloc[:,0:8], axis=1, inplace=True)
del electricity_fr['OBS_FLAG']
electricity_fr['TIME_PERIOD'] = electricity_fr['TIME_PERIOD'].apply(lambda x : str(x) + '-01-01')
electricity_fr['Date_index'] = pd.to_datetime(electricity_fr['TIME_PERIOD'])
electricity_fr = electricity_fr.loc[(electricity_fr['Date_index'] >'2012-01-01')]
electricity_fr['year'] = electricity_fr['Date_index'].dt.year


In [ ]:
master = pd.merge(master, electricity_fr, how='left', on='year')

# Create & merge gas dataset

In [ ]:
gas = pd.read_csv('/Users/ymilo/Desktop/Jedha/Final_Project/prix_gaz_naturel.csv', sep=';')
gas['Date_index'] = pd.to_datetime(gas['Date'])
gas['year'] = gas['Date_index'].dt.year
gas['month'] = gas['Date_index'].dt.month

In [ ]:
master['gas_key'] = master['year'].astype(str) + master['month'].astype(str)
gas['gas_key'] = gas['year'].astype(str) + gas['month'].astype(str)

In [ ]:
del gas['Date']
del gas['Date_index']
del gas['year']
del gas['month']

In [ ]:
master = pd.merge(master, gas, on='gas_key', how='left')

# Tiddy up master

In [ ]:
master['Date'] = pd.to_datetime(master['Date'])

In [ ]:
master['day'] = master['Date'].dt.day

In [ ]:
master['day_of_week'] = master['Date'].dt.day_of_week

In [ ]:
master = master.rename(columns={'Europe Brent Spot Price FOB (Dollars per Barrel)':'brent_price' , 'OBS_VALUE':'prix_kwh_elec', 'PEG':'prix_gaz'})

In [ ]:
del master['Date_index_x']
del master['Date_index_y']
del master['gas_key']

In [ ]:
# Add a column with corresponding region names for each region code

dic_code_reigions = {11 : 'IDF', 
24 : 'Centre', 
27 : 'Bourgogne', 
28 : 'Normandie', 
32 : 'Hauts-de-France', 
44 : 'Grand Est', 
52 : 'Loire',
75 : 'Aquitaine',
76 : 'Occitanie',
93 : 'Provence',
84 : 'Auvergen',
53 : 'Bretagne'}

master['Nom_region'] = master['Code INSEE région'].map(dic_code_reigions)

_column = master.pop('Nom_region')

master.insert(2, 'Nom_region', _column)

In [ ]:
# Create copy of master dataset for machine learning
master_ml = master.copy()

In [ ]:
del master_ml['Thermique (MW)']
del master_ml['Nucléaire (MW)']
del master_ml['Eolien (MW)']
del master_ml['Solaire (MW)']
del master_ml['Hydraulique (MW)']
del master_ml['Bioénergies (MW)']
del master_ml['Ech. physiques (MW)']
del master_ml['Stockage batterie']

In [ ]:
# Remove row with outlier
master_final = master[master['Date'] != '2022-10-30']

# Add lag to datasets

In [ ]:
# Function to create lag from target 'Consommation (MW)'
def lag(x):
    for i in range(15):
        x[f'lag_{i+1}'] = x['Consommation (MW)'].shift(i+1)
    x['rolling_mean_7'] = x['Consommation (MW)'].rolling(window=7).mean()
    x['rolling_mean_15'] = x['Consommation (MW)'].rolling(window=15).mean()
    x['lag_364'] = x['Consommation (MW)'].shift(364)
    for i in range(1, 8):
        x[f'lag_inversed_{i}'] = x['Consommation (MW)'].shift(-i)
    x = x.dropna(subset=['lag_364'])

    return x

# Function to seperate dataset by region, lag it and concat it
def concator(x):
    reg_codes = x['Code INSEE région'].unique()
    data_list = []
    lag_data_list = []
    for i in reg_codes:
        data_list.append(x[x['Code INSEE région'] == i])
    for i in range(len(data_list)):
        lag_data_list.append(lag(data_list[i]))
    final = pd.concat(lag_data_list)
    return final


In [ ]:
master_lag = concator(master_final)

In [ ]:
master_lag_ml = concator(master_ml)

In [ ]:
# Reset index to use date
master_lag = master_lag.set_index('Date')
master_lag_ml = master_lag_ml.set_index('Date')

In [ ]:
master_lag = master_lag.rename(columns = {'Code INSEE région' : 'code_region'})
master_lag_ml = master_lag_ml.rename(columns = {'Code INSEE région' : 'code_region'})

In [ ]:
master_lag_ml.to_csv('/Users/ymilo/Desktop/Jedha/Final_Project/master_lag_ml_inversed.csv')